[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/trainNNmodel.ipynb)

In [1]:
!pip install tqdm

In [2]:
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers

from tqdm import tqdm

Using TensorFlow backend.


In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/
%ls

/content/gdrive/My Drive/projects
model_LGB.sav  model_XGB.sav  user_train_wip.csv


In [0]:
s_data = 'user_train_wip.csv'
s_col_tgt = 'totals.transactionRevenue_sum'
l_col_drop = ['date_min', 'date_max',s_col_tgt]
filename = 'model_NN.sav'

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 10.5 s, sys: 1.76 s, total: 12.3 s
Wall time: 14.9 s


In [7]:
%%time
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_x_n = pd.DataFrame(sc.fit_transform(df_x.fillna(0)),columns=df_x.columns)
df_x_n

CPU times: user 2.53 s, sys: 1.14 s, total: 3.67 s
Wall time: 3.73 s


In [0]:
def build_model(N_col):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(N_col,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [14]:
%%time
epochs = 10
batch_size = 200000

model = build_model(df_x.shape[1])

history = model.fit(df_x_n,
                    df_y,
                    epochs=epochs,
                    batch_size=batch_size,
                    verbose=1)

Epoch 1/10
714167/714167 [==============================] - 4s 5us/step - loss: 8.5647 - mean_absolute_error: 0.6081
Epoch 2/10
714167/714167 [==============================] - 4s 5us/step - loss: 7.4620 - mean_absolute_error: 0.5780
Epoch 3/10
714167/714167 [==============================] - 4s 5us/step - loss: 6.9345 - mean_absolute_error: 0.6057
Epoch 4/10
714167/714167 [==============================] - 4s 5us/step - loss: 6.5643 - mean_absolute_error: 0.6067
Epoch 5/10
714167/714167 [==============================] - 4s 5us/step - loss: 6.2707 - mean_absolute_error: 0.6000
Epoch 6/10
714167/714167 [==============================] - 4s 5us/step - loss: 6.0583 - mean_absolute_error: 0.5919
Epoch 7/10
714167/714167 [==============================] - 3s 5us/step - loss: 5.9061 - mean_absolute_error: 0.5818
Epoch 8/10
714167/714167 [==============================] - 4s 5us/step - loss: 5.7582 - mean_absolute_error: 0.5593
Epoch 9/10
714167/714167 [==============================] - 4s 5

In [15]:
history.history

{'loss': [8.564659477529949,
  7.46198294669111,
  6.934467788110956,
  6.564304266775209,
  6.270706642836594,
  6.058307173662174,
  5.906127979676258,
  5.7581761899735096,
  5.755019950471297,
  5.652122472863003],
 'mean_absolute_error': [0.6080875236581412,
  0.5780460474334104,
  0.6057166252831829,
  0.6067252476955787,
  0.5999682404565487,
  0.5918505922535273,
  0.5818488258317185,
  0.5593260598749341,
  0.5811313638354567,
  0.5535816328732629]}

In [0]:
%%time
pickle.dump(model, open(filename, 'wb'))

In [0]:
files.download(filename)